In [1]:
import pandas as pd
import json
import requests
import glob
import os
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

# Hackathon #2 - Data Wrangling (Instructor solution)

## Merge the Data

We'll be concatenating all the dataframes, droping any duplicated entries and observations with missing data

In [2]:
pd.read_csv('file_data.csv', index_col='id').sort_index().iloc[[-1]].index

Int64Index([10000], dtype='int64', name='id')

In [3]:
pd.read_csv('database_data.csv', index_col='id').sort_index().iloc[[-1]].index

Int64Index([9999], dtype='int64', name='id')

In [4]:
pd.read_csv('website_data.csv', index_col='id').sort_index().iloc[[-1]].index

Int64Index([9994], dtype='int64', name='id')

In [9]:
df = pd.concat([
    pd.read_csv('api_data.csv', index_col='id'),
    pd.read_csv('file_data.csv', index_col='id'),
    pd.read_csv('database_data.csv', index_col='id'),
    pd.read_csv('website_data.csv', index_col='id')
]).sort_index()
df

,submitinfotoemail,iframeorframe,missingtitle,imagesonlyinform,subdomainlevelrt,urllengthrt,pctextresourceurlsrt,abnormalextformactionr,extmetascriptlinkrt,pctextnullselfredirecthyperlinksrt,...,numquerycomponents,insecureforms,urllength,pctnullselfredirecthyperlinks,numdots,frequentdomainnamemismatch,numsensitivewords,pathlength,extformaction,pctexthyperlinks
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,72.0,0.0,3.0,0.0,0.0,44.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,144.0,0.0,3.0,0.0,1.0,16.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,49.0,0.0,3.0,0.0,0.0,13.0,0.0,0.0
9999,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# so lets fill the null values (within the same column and the same id) 
# with any non null value for the same id
df = df.fillna(method='ffill').fillna(method='bfill').sort_index()
df

,submitinfotoemail,iframeorframe,missingtitle,imagesonlyinform,subdomainlevelrt,urllengthrt,pctextresourceurlsrt,abnormalextformactionr,extmetascriptlinkrt,pctextnullselfredirecthyperlinksrt,...,numquerycomponents,insecureforms,urllength,pctnullselfredirecthyperlinks,numdots,frequentdomainnamemismatch,numsensitivewords,pathlength,extformaction,pctexthyperlinks
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,72.0,0.0,3.0,0.0,0.0,44.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,72.0,0.0,3.0,0.0,0.0,44.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,72.0,0.0,3.0,0.0,0.0,44.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,...,2.0,1.0,144.0,0.0,3.0,0.0,1.0,16.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,2.0,1.0,144.0,0.0,3.0,0.0,1.0,16.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,57.0,0.0,2.0,0.0,0.0,34.0,0.0,0.0
9999,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,49.0,0.0,3.0,0.0,0.0,13.0,0.0,0.0
9999,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,1.0,1.0,49.0,0.0,3.0,0.0,0.0,13.0,0.0,0.0


In [11]:
# then keep only 1 row per id
df = df.groupby(df.index).first()
df

,submitinfotoemail,iframeorframe,missingtitle,imagesonlyinform,subdomainlevelrt,urllengthrt,pctextresourceurlsrt,abnormalextformactionr,extmetascriptlinkrt,pctextnullselfredirecthyperlinksrt,...,numquerycomponents,insecureforms,urllength,pctnullselfredirecthyperlinks,numdots,frequentdomainnamemismatch,numsensitivewords,pathlength,extformaction,pctexthyperlinks
id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,72.0,0.0,3.0,0.0,0.0,44.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,72.0,0.0,3.0,0.0,0.0,44.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,2.0,1.0,144.0,0.0,3.0,0.0,1.0,16.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,58.0,0.0,3.0,0.0,0.0,24.0,0.0,0.0
5,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,79.0,0.0,3.0,0.0,0.0,50.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,2.0,1.0,50.0,0.0,3.0,0.0,0.0,10.0,0.0,0.0
9997,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,2.0,1.0,50.0,0.0,3.0,0.0,0.0,10.0,0.0,0.0
9998,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,57.0,0.0,2.0,0.0,0.0,34.0,0.0,0.0


## Create Train Dataset

In [12]:
df.to_csv("merge_data.csv")